<a href="https://colab.research.google.com/github/chirazedrine/EmoGen/blob/main/EmoGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0: Dowload dependencies

In [ ]:
!pip install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.3 MB/s eta 0:00:00


# Step 1: Download and extract the dataset

In [ ]:
!wget -O EmoSet-118K.zip "https://www.dropbox.com/scl/fi/myue506itjfc06m7svdw6/EmoSet-118K.zip?dl=1&rlkey=7f3oyjkr6zyndf0gau7t140rv"


--2024-03-19 18:51:04--  https://www.dropbox.com/scl/fi/myue506itjfc06m7svdw6/EmoSet-118K.zip?dl=1&rlkey=7f3oyjkr6zyndf0gau7t140rv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd979efa0cdc8e133e88ec69690.dl.dropboxusercontent.com/cd/0/inline/CPZ8aafbmgj_XIkwJshLXXid56aHdpEjdCLCQPnYmAip7RZjUUjHKcHglrn56cvvVOspHrnDIJTi3uys8KWorw3u0MfIhe2TU8B21LfKrIEjE8ckva94oQHrCK6Qf3x4pMa4N3wzw4nZH_9rj3mebmX0/file?dl=1# [following]
--2024-03-19 18:51:05--  https://ucd979efa0cdc8e133e88ec69690.dl.dropboxusercontent.com/cd/0/inline/CPZ8aafbmgj_XIkwJshLXXid56aHdpEjdCLCQPnYmAip7RZjUUjHKcHglrn56cvvVOspHrnDIJTi3uys8KWorw3u0MfIhe2TU8B21LfKrIEjE8ckva94oQHrCK6Qf3x4pMa4N3wzw4nZH_9rj3mebmX0/file?dl=1
Resolving ucd979efa0cdc8e133e88ec69690.dl.dropboxusercontent.com (ucd979efa0cdc8e133e88ec69690.dl.dropboxusercontent.com)... 

In [ ]:
!unzip -q EmoSet-118K.zip

In [ ]:
!mkdir -p /content/EmoSet-118K  # Create the target directory if it doesn't exist
!mv /content/annotation /content/EmoSet-118K/
!mv /content/image /content/EmoSet-118K/
!mv /content/*.json /content/EmoSet-118K/  # Move all json files


# Step 2: Parse the JSON files to create a mapping for image paths and labels

In [ ]:
import os
import json
import torch
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch.nn as nn

# Define directories
base_path = '/content/EmoSet-118K'
annotations_dir = os.path.join(base_path, 'annotation')
images_dir = os.path.join(base_path, 'image')

# Define a function to parse JSON annotations
def parse_annotations(annotations_dir):
    annotations = []
    for emotion_dir_name in os.listdir(annotations_dir):
        emotion_dir_path = os.path.join(annotations_dir, emotion_dir_name)
        if os.path.isdir(emotion_dir_path):
            for annotation_file in os.listdir(emotion_dir_path):
                annotation_file_path = os.path.join(emotion_dir_path, annotation_file)
                with open(annotation_file_path, 'r') as f:
                    annotation_data = json.load(f)
                    annotations.append(annotation_data)
    return annotations

annotations = parse_annotations(annotations_dir)

# Verify the number of annotations loaded
print(f"Loaded {len(annotations)} annotations.")

# Extract unique emotions and create a mapping to indices
unique_emotions = sorted({anno['emotion'] for anno in annotations})
emotion_to_idx = {emotion: idx for idx, emotion in enumerate(unique_emotions)}
print(f"Unique emotions found: {list(emotion_to_idx.keys())}")

# Custom Dataset class
class EmoSetDataset(Dataset):
    def __init__(self, annotations, images_dir, transform=None):
        self.annotations = annotations
        self.images_dir = images_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        annotation = self.annotations[idx]
        image_path = os.path.join(self.images_dir, annotation['emotion'], f"{annotation['image_id']}.jpg")
        image = Image.open(image_path).convert('RGB')
        label = emotion_to_idx[annotation['emotion']]

        if self.transform:
            image = self.transform(image)

        return image, label

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Dataset and DataLoader
dataset = EmoSetDataset(annotations, images_dir, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)


Loaded 118102 annotations.
Unique emotions found: ['amusement', 'anger', 'awe', 'contentment', 'disgust', 'excitement', 'fear', 'sadness']


# Step 3: Define the CNN model for emotion classification

In [ ]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(unique_emotions))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Step 4: Define loss function and optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Check if the data loader can fetch data
for images, labels in data_loader:
    print(f"Fetched {images.shape[0]} samples.")
    break  # Only fetch one batch for checking


Fetched 32 samples.


# Step 5: Train the model

In [ ]:
num_epochs = 5  # Define the number of epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0

    for inputs, labels in data_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_predictions += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(data_loader.dataset)
    epoch_acc = correct_predictions.double() / len(data_loader.dataset)

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

print('Finished Training')

# Step 6: Evaluate the model

In [ ]:
model_path = '/content/drive/My Drive/emo_model.pth'
torch.save(model.state_dict(), model_path)

# Reload the model weights
model.load_state_dict(torch.load(model_path))

# Evaluate on the test set
test_annotations = [anno for anno in annotations.values() if anno['split'] == 'test']
test_dataset = EmoSetDataset(test_annotations, images_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
running_corrects = 0

for inputs, labels in test_loader:
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    running_corrects += torch.sum(preds == labels.data)

test_acc = running_corrects.double() / len(test_loader.dataset)
print(f'Test Accuracy: {test_acc:.4f}')